In [10]:
import pyvista as pv
import glob
import laspy as lp
import numpy as np

In [11]:
main_path = '/home/diego/Documents/2023-05-24_Kinematic_Vitacura/LAZ - 1cm/*'
glob.glob(main_path)

['/home/diego/Documents/2023-05-24_Kinematic_Vitacura/LAZ - 1cm/230518_085049_001.laz',
 '/home/diego/Documents/2023-05-24_Kinematic_Vitacura/LAZ - 1cm/230518_085049.laz',
 '/home/diego/Documents/2023-05-24_Kinematic_Vitacura/LAZ - 1cm/230518_085049_002.laz']

In [12]:
def get_points_and_colors(las_file, limit=-1):
    las = lp.read(las_file)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    colors = np.vstack((las.red, las.green, las.blue)).transpose()
    if limit > 0:
        idxs = np.random.choice(len(points), limit, replace=False)
        points = points[idxs]
        colors = colors[idxs]
    # Another way
    #factor=10
    #decimated_points_random = points[::factor]
    return points, colors

In [13]:
path_1 = '/home/diego/Documents/2023-05-24_Kinematic_Vitacura/LAZ - crudo/230518_085049_001.laz'
path_2 = '/home/diego/Documents/2023-05-24_Kinematic_Vitacura/LAZ - 1cm/230518_085049_001.laz'

In [14]:
points_1, colors_1 = get_points_and_colors(path_1, limit=100000)
points_2, colors_2 = get_points_and_colors(path_2, limit=100000)

In [17]:
point_cloud_1 = pv.PolyData(points_1)
point_cloud_2 = pv.PolyData(points_2)
point_cloud_1, point_cloud_2

(PolyData (0x75f65b025360)
   N Cells:    100000
   N Points:   100000
   N Strips:   0
   X Bounds:   3.514e+05, 3.515e+05
   Y Bounds:   6.303e+06, 6.304e+06
   Z Bounds:   6.691e+02, 7.045e+02
   N Arrays:   0,
 PolyData (0x75f65b0259c0)
   N Cells:    100000
   N Points:   100000
   N Strips:   0
   X Bounds:   3.514e+05, 3.515e+05
   Y Bounds:   6.303e+06, 6.304e+06
   Z Bounds:   6.685e+02, 6.981e+02
   N Arrays:   0)

In [18]:
surf_1 = point_cloud_1.delaunay_2d()
surf_2 = point_cloud_2.delaunay_2d()

In [8]:
#surf_1.flip_normals()
#surf_2.flip_normals()

In [19]:
surf_1.is_all_triangles, surf_2.is_all_triangles

(True, True)

In [20]:
#surf_1.compute_normals(inplace=True)
#surf_1.plot_normals()

In [21]:
plotter = pv.Plotter()
plotter.add_mesh(surf_1, show_edges=False, color='red')
plotter.add_mesh(surf_2, show_edges=False, color='blue')
#plotter.add_mesh(intersection, show_edges=False, color='green')
plotter.show()

Widget(value='<iframe src="http://localhost:40183/index.html?ui=P_0x75f65c231bb0_0&reconnect=auto" class="pyvi…

## Distance Between Two Surfaces

### 1. Nearest Neighbor Distance (rápida)

In [22]:
from scipy.spatial import KDTree

tree = KDTree(surf_2.points)
d_kdtree, idx = tree.query(surf_1.points)
surf_1["distances"] = d_kdtree
np.mean(d_kdtree)

0.2591295929118971

In [23]:
p = pv.Plotter()
p.add_mesh(surf_1, scalars="distances", smooth_shading=True)
p.add_mesh(surf_2, color=True, opacity=0.75, smooth_shading=True)
p.show()

Widget(value='<iframe src="http://localhost:40183/index.html?ui=P_0x75f6517c3da0_1&reconnect=auto" class="pyvi…

### 2. Ray Tracing Distance (más lento, lo paré a los 16 minutos, que onda?)

In [47]:
surf_1_n = surf_1.compute_normals(point_normals=True, cell_normals=False, auto_orient_normals=True)

In [49]:
surf_1_n["distances"] = np.empty(surf_1.n_points)
for i in range(surf_1_n.n_points):
    p = surf_1_n.points[i]
    vec = surf_1_n["Normals"][i] * surf_1_n.length
    p0 = p - vec
    p1 = p + vec
    ip, ic = surf_2.ray_trace(p0, p1, first_point=True)
    dist = np.sqrt(np.sum((ip - p) ** 2))
    surf_1_n["distances"][i] = dist

# Replace zeros with nans
mask = surf_1_n["distances"] == 0
surf_1_n["distances"][mask] = np.nan
np.nanmean(surf_1_n["distances"])

KeyboardInterrupt: 

: 

### 3. Using PyVista Filter

In [24]:
closest_cells, closest_points = surf_2.find_closest_cell(surf_2.points, return_closest_point=True)
d_exact = np.linalg.norm(surf_1.points - closest_points, axis=1)
surf_1["distances"] = d_exact
np.mean(d_exact)

145.55014683036833

In [25]:
p = pv.Plotter()
p.add_mesh(surf_1, scalars="distances", smooth_shading=True)
p.add_mesh(surf_2, color=True, opacity=0.75, smooth_shading=True)
p.show()

Widget(value='<iframe src="http://localhost:40183/index.html?ui=P_0x75f640c62390_2&reconnect=auto" class="pyvi…

In [9]:
# Make data array using z-component of points array
#data_1 = points_1[:, -1]
#data_2 = points_2[:, -1]

# Add that data to the mesh with the name "uniform dist"
#point_cloud_1["elevation"] = data_1
#point_cloud_2["elevation"] = data_2

In [14]:
intersection = point_cloud_1.boolean_intersection(point_cloud_2)

NotAllTrianglesError: Make sure both the input and output are triangulated.

In [12]:
opacity = 0.25
point_size = 5

In [13]:
plotter = pv.Plotter()
plotter.add_points(point_cloud_1, opacity=opacity, point_size=point_size, render_points_as_spheres=True, color='red')
plotter.add_points(point_cloud_2, opacity=opacity, point_size=point_size, render_points_as_spheres=True, color='blue')
plotter.show()

Widget(value='<iframe src="http://localhost:45055/index.html?ui=P_0x733bfc855370_1&reconnect=auto" class="pyvi…

In [9]:
#point_cloud.plot(eye_dome_lighting=True, render_points_as_spheres=True, point_size=point_size, opacity=opacity)

Widget(value='<iframe src="http://localhost:41057/index.html?ui=P_0x7f5689ea2f30_1&reconnect=auto" class="pyvi…